In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import jax
print(jax.devices())

In [ ]:
import sys

sys.path.append('..')

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def print_grid_limits(dx, c=343.0, total_time=None):
    # 1. Spatial Limit (10 points per wavelength)
    # This is the "High Frequency Ceiling"
    f_spatial_limit = c / (10 * dx)
    
    print(f"--- Grid Physics Limits (dx={dx}m, c={c}m/s) ---")
    print(f"Max Accurate Freq (10 PPW): {f_spatial_limit:.1f} Hz")
    print(f"Max Usable Freq (4 PPW - limit): {c / (4 * dx):.1f} Hz")
    
    if total_time:
        # 2. Spectral Limit (DFT bin width)
        df = 1.0 / total_time
        print(f"Frequency Bin Resolution:   {df:.2f} Hz")

# Example usage with your current settings:
print_grid_limits(dx=0.01, c=343.0, total_time=0.5)

### Creating Boundary Conditions by Masking

In [ ]:
import numpy as np
from src.core import Domain2D
from src.solvers import Wave, WaveJAX
from src.components import RickerSource, Listener
from src.visualization import PhysicsAnimator
import src.utils as utils

# 1. Setup Scene
room = Domain2D(length=[10, 10], dx=0.02, R=0.8)

# 2. Populate Scene (Geometry + Source + Material)
# room.add_smart_speaker(
#     center=[5, 3], 
#     source = RickerSource(pos=[5, 3], peak_freq=2000.0, delay=0.02), 
#     inner_size=[1, 1]
#     )
# room.add_source(RickerSource(pos=[5, 3], peak_freq=2000.0, delay=0.02))

# 3. Add a Microphone
room.add_listener(Listener(pos=[5, 8])) # In front
room.add_listener(Listener(pos=[8, 2])) # To the side (Shadow)

# 4. DEBUG: Check coordinates instantly!
room.preview() 
# ^ If there are gaps in the box, you will see them here immediately.

# 5. Run Physics
# The Solver now pulls everything from the domain
solver = WaveJAX(domain=room, initial_u = utils.get_initial_gaussian(pos=[5,5], sigma=0.1), c=343.0)

In [ ]:
import time
start_time = time.time()
animator = PhysicsAnimator(solver = solver, total_time=0.5)
animator.run()
print(f"Simulation Time: {time.time() - start_time:.2f} seconds")

In [ ]:
animator.create_animation(skip_frames=10, skip_spatial=5)

In [ ]:
plt.plot(*solver.domain.listeners[0].get_time_series(), alpha = 0.6)
plt.plot(*solver.domain.listeners[1].get_time_series(), alpha = 0.6)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_frequency_response(listener, label="Listener"):
    # 1. Get Data
    freqs, mag = listener.compute_spectrum()
    
    if freqs is None:
        print("Not enough data to plot spectrum.")
        return

    # 2. Convert to Decibels (dB)
    # add a tiny epsilon (1e-12) to avoid log(0) errors
    mag_db = 20 * np.log10(mag + 1e-12)
    
    # Normalize to peak (optional, makes 0dB the max volume)
    mag_db -= np.max(mag_db)

    # 3. Plot
    plt.plot(freqs, mag_db, label=label, linewidth=1.5)
    
    # 4. Make it look like Audio
    plt.xscale('log')  # <--- CRITICAL for audio
    plt.grid(True, which="both", ls="-", alpha=0.5)
    plt.xlim(20, 20000) # Standard hearing range (20Hz - 20kHz)
    
    # Optional: Cut off the bottom noise floor for cleaner look
    plt.ylim(-60, 5) 

    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Magnitude (dB)')
    plt.title('Frequency Response')
    plt.legend()

# Usage:
plt.figure(figsize=(10, 6))

# Plot all listeners
for i, listener in enumerate(solver.domain.listeners):
    plot_frequency_response(listener, label=f"Mic {i+1}")

plt.show()